##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
sys.path.append('../') # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-search"></i> 查找InvokeAI返回项目所在路径

In [ ]:
import os
from func.checksum import generate_checksum,find_folder

def find_dir(root_dir, target_name):
    paths = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for dirname in dirnames:
            if dirname == target_name:
                path = os.path.join(dirpath, dirname)
                paths.append(path)
    paths.sort(key=len)
    if paths:
        return paths
    else:
        return None

# print(generate_checksum('/root/InvokeAI'))
# print(generate_checksum('/root/InvokeAI_runtime'))

# 通过校验和的方式查找InvokeAI的github repo文件夹
InvokeAI_dir = None
# InvokeAI_dir = find_folder(root_dir,'827a2072f6f785ae0abf58be160b69b0e165b9a28eb2a69bae426eb268f90127')

# 如果找不到，则采用低阶的办法，也就是文件夹名称完全匹配的版本进行查找（因为repo总是更新，会影响校验和的结果）
if not InvokeAI_dir:
    InvokeAI_dir=find_dir(root_dir, 'InvokeAI')[0]
    
print(InvokeAI_dir)
    
InvokeAI_runtime_dir = None
# InvokeAI_runtime_dir = find_folder(root_dir,'c7dd5c092639d9ac32ea5d1bbcaa9ec4ed9ade22d2c0c5621d395af5c1cda267')
if not InvokeAI_runtime_dir:
    InvokeAI_runtime_dir=find_dir(root_dir, 'InvokeAI_runtime')[0]

print(InvokeAI_runtime_dir)

## <i class="fa fa-hammer"></i> 配置invokeai.init文件
通过修改这里的参数来自定义invokeai的配置，在启动invokeai的时候会传入对应的参数，如修改--outdir来改变invokeai生成的目标路径，修改--port来修改运行端口

In [ ]:
port = 6006

if env_name == 'AutoDL':
    invoke_outputs_dir='/root/autodl-nas/invoke_outputs'
    invokeai_init_path=os.path.join(InvokeAI_runtime_dir,'./invokeai.init')
elif env_name == 'OpenBayes':
    invoke_outputs_dir=os.path.realpath(os.path.join(root_dir,'./invoke_outputs'))
    invokeai_init_path=os.path.realpath(os.path.join(root_dir,'./invokeai/invokeai.init'))
    
import re

def add_line(lines, line_to_add):
    # 如果对应的参数和文本不存在，在文件末尾增加一行
    if not any(line.strip() == f'{line_to_add}' for line in lines):
        lines.append(f'{line_to_add}\n')

# 读取文件
with open(invokeai_init_path, 'r') as f:
    lines = f.readlines()

# 进行文本替换
for i, line in enumerate(lines):
    lines[i] = re.sub(r'--outdir="(.*)"', f'--outdir="{invoke_outputs_dir}"', line)

#追加额外的参数
add_line(lines,'--no-nsfw_checker')
add_line(lines,'--no-safety_checker')
add_line(lines,f'--port={port}') #设定端口
add_line(lines,'--web') #以webui的形式启动

# 去除重复的行
seen = {}
result = []
for line in lines:
    if line in seen:
        continue
    seen[line] = True
    result.append(line)

lines = result
    
# 去除重复的行，但是这种做法会导致行序打乱
# lines = list(set(lines))

# 将修改后的内容写回文件
with open(invokeai_init_path, 'w') as f:
    f.writelines(lines)


## <i class="fa fa-eye"></i> 以WebUI的形式去启动InvokeAI

In [ ]:
base_args = [
    proxy,'&&',
    'source activate','&&',
    'conda activate invokeai','&&',
    f'cd {InvokeAI_dir}','&&'
]

final_args = []
final_args += base_args

git_pull_args = base_args[:] + ['git pull']
git_pull_cmd = (" ").join(git_pull_args)

git_pull_info = !$git_pull_cmd

print(git_pull_info)

if 'Already up to date.' not in git_pull_info:
    print('准备更新...')
    final_args.append([
        'conda env update','&&',
        f'python scripts/configure_invokeai.py -y --skip-sd-weights --root_dir {InvokeAI_runtime_dir}'
    ])
else:
    print('已经更新到最新，直接启动')
    
'''
在 Python 中，可以使用以下几种方法来拼接两个列表：
使用 + 操作符：new_list = list1 + list2。这会创建一个新的列表，其中包含原始两个列表中的所有元素。
使用内置函数 extend()：list1.extend(list2)。这会将 list2 中的所有元素添加到 list1 的末尾。
使用内置函数 sum()：new_list = sum([list1, list2], [])。这会将两个列表拼接起来，并将结果保存到新的列表中。
'''

final_args = base_args[:]
final_args += [
    f'python scripts/invoke.py --root_dir {InvokeAI_runtime_dir}',
]

final_cmd = (" ").join(final_args)
print(final_cmd)
!$final_cmd